<a href="https://colab.research.google.com/github/ananyapandey1/ADS-Lab/blob/main/ADS_EXP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import nltk
import re
import warnings

warnings.filterwarnings("ignore")

pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')
pd.set_option('display.max_columns', 200)

nltk.download('brown')
nltk.download('punkt')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
movies_df = pd.read_csv("/content/movies.csv")
movies_df.head()

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced into action when a group of terrorists attempt to hijack a transatlantic overnight flight.,"\n Director:\nPeter Thorwarth\n| \n Stars:\nPeri Baumeister, \nCarl Anton Koch, \nAlexander Scheer, \nKais Setti\n","21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may be the final battle between He-Man and Skeletor. A new animated series from writer-director Kevin Smith.,"\n \n Stars:\nChris Wood, \nSarah Michelle Gellar, \nLena Headey, \nMark Hamill\n","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a coma to learn the world is in ruins and must lead a group of survivors to stay alive.,"\n \n Stars:\nAndrew Lincoln, \nNorman Reedus, \nMelissa McBride, \nLauren Cohan\n","885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits of a super scientist and his not-so-bright grandson.,"\n \n Stars:\nJustin Roiland, \nChris Parnell, \nSpencer Grammer, \nSarah Chalke\n","414,849",23.0,NaN
4,Army of Thieves,(2021),"\nAction, Crime, Horror",NaN,"\nA prequel, set before the events of Army of the Dead, which focuses on German safecracker Ludwig Dieter leading a group of aspiring thieves on a top secret heist during the early stages of the zombie apocalypse.","\n Director:\nMatthias Schweighöfer\n| \n Stars:\nMatthias Schweighöfer, \nNathalie Emmanuel, \nRuby O. Fee, \nStuart Martin\n",NaN,NaN,NaN


In [ ]:
movies_df.dtypes

MOVIES       object
YEAR         object
GENRE        object
RATING      float64
ONE-LINE     object
STARS        object
VOTES        object
RunTime     float64
Gross        object
dtype: object

In [ ]:
movies_df.shape

(9999, 9)

In [ ]:
df_columns = movies_df.columns.tolist()
df_columns

['MOVIES',
 'YEAR',
 'GENRE',
 'RATING',
 'ONE-LINE',
 'STARS',
 'VOTES',
 'RunTime',
 'Gross']

In [ ]:
for c in df_columns:
    movies_df = movies_df.rename(columns=({c: movies_df[c].name.title()}))

movies_df = movies_df.rename(columns=({'Gross': 'Gross (in million $)'}))

movies_df.columns.tolist()

['Movies',
 'Year',
 'Genre',
 'Rating',
 'One-Line',
 'Stars',
 'Votes',
 'Runtime',
 'Gross (in million $)']

In [ ]:
movies_df = movies_df.drop_duplicates()
movies_df.shape

(9568, 9)

In [ ]:
movies_df['Movies'] = movies_df['Movies'].str.strip()
movies_df['Movies']

0                             Blood Red Sky
1       Masters of the Universe: Revelation
2                          The Walking Dead
3                            Rick and Morty
4                           Army of Thieves
                       ...                 
9993                              Totenfrau
9995                                 Arcane
9996                      Heart of Invictus
9997                         The Imperfects
9998                         The Imperfects
Name: Movies, Length: 9568, dtype: object

In [ ]:
movies_df['Votes'] = movies_df['Votes'].fillna('0')
movies_df['Votes'] = movies_df['Votes'].str.replace(',', '')
movies_df['Votes'] = movies_df['Votes'].astype(int)

In [ ]:
movies_df['Rating'] = movies_df['Rating'].fillna(0.0)

In [ ]:
movies_df['Gross (in million $)'] = movies_df['Gross (in million $)'].str.replace('M', '').str.replace('$', '')
movies_df['Gross (in million $)'] = movies_df['Gross (in million $)'].astype(float)
movies_df['Gross (in million $)'] = movies_df['Gross (in million $)'].fillna(0.0)

In [ ]:
movies_df['Runtime'] = movies_df['Runtime'].fillna(0.0)

In [ ]:
#Split year into start year and End year
movies_df["Year"] = movies_df["Year"].str.replace('(', '')
movies_df["Year"] = movies_df["Year"].str.replace(')', '')
movies_df['Year'] = movies_df['Year'].str.strip()
movies_df[["Mix", "End_Year"]] = movies_df['Year'].str.split('–', 1, expand=True)
# Split the Start_Year column
movies_df['Start_Year'] = movies_df['Mix'].astype(str).apply(lambda x: ''.join(re.findall(r'\d{4}', x)))
movies_df['Season'] = movies_df['Mix'].astype(str).apply(lambda x: ''.join(re.findall(r'\b[I|V|X|L|C|D|M]+\b', x)))
movies_df['Type'] = movies_df['Mix'].astype(str).apply(lambda x: ''.join(re.sub(r'(\d{4}|\b[I|V|X|L|C|D|M]+\b)', '', x)))
movies_df.head()

,Movies,Year,Genre,Rating,One-Line,Stars,Votes,Runtime,Gross (in million $),Mix,End_Year,Start_Year,Season,Type
0,Blood Red Sky,2021,"\nAction, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced into action when a group of terrorists attempt to hijack a transatlantic overnight flight.,"\n Director:\nPeter Thorwarth\n| \n Stars:\nPeri Baumeister, \nCarl Anton Koch, \nAlexander Scheer, \nKais Setti\n",21062,121.0,0.0,2021,None,2021,,
1,Masters of the Universe: Revelation,2021–,"\nAnimation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may be the final battle between He-Man and Skeletor. A new animated series from writer-director Kevin Smith.,"\n \n Stars:\nChris Wood, \nSarah Michelle Gellar, \nLena Headey, \nMark Hamill\n",17870,25.0,0.0,2021,,2021,,
2,The Walking Dead,2010–2022,"\nDrama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a coma to learn the world is in ruins and must lead a group of survivors to stay alive.,"\n \n Stars:\nAndrew Lincoln, \nNorman Reedus, \nMelissa McBride, \nLauren Cohan\n",885805,44.0,0.0,2010,2022,2010,,
3,Rick and Morty,2013–,"\nAnimation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits of a super scientist and his not-so-bright grandson.,"\n \n Stars:\nJustin Roiland, \nChris Parnell, \nSpencer Grammer, \nSarah Chalke\n",414849,23.0,0.0,2013,,2013,,
4,Army of Thieves,2021,"\nAction, Crime, Horror",0.0,"\nA prequel, set before the events of Army of the Dead, which focuses on German safecracker Ludwig Dieter leading a group of aspiring thieves on a top secret heist during the early stages of the zombie apocalypse.","\n Director:\nMatthias Schweighöfer\n| \n Stars:\nMatthias Schweighöfer, \nNathalie Emmanuel, \nRuby O. Fee, \nStuart Martin\n",0,0.0,0.0,2021,None,2021,,


In [ ]:
movies_df['Genre'] = movies_df['Genre'].str.strip('\n')
movies_df[['Genre1', 'Genre2', 'Genre3']] = movies_df['Genre'].str.split(',',2,expand=True)

In [ ]:
#Split and clean the Stars column from the dataframe
movies_df['Stars'] = movies_df['Stars'].str.strip('\n')
stars = pd.DataFrame()
movies_df[['Director', 'Actor']] = movies_df['Stars'].str.split('|', 1, expand=True)

for idx, value in movies_df['Director'].iteritems():
    #if idx < 10: print(idx, value)
    if 'Stars:' in value:
        movies_df['Actor'][idx], movies_df['Director'][idx] = value, ''

movies_df['Director'] = movies_df['Director'].str.replace('Director:', '')
movies_df['Director'] = movies_df['Director'].str.replace('Directors:', '')
movies_df['Actor'] = movies_df['Actor'].str.replace('Stars:', '')
movies_df['Actor'] = movies_df['Actor'].str.replace('Star:', '')

movies_df['Director'] = movies_df['Director'].str.strip()
movies_df['Actor'] = movies_df['Actor'].str.strip()
movies_df['Actor'] = movies_df['Actor'].str.replace('\n', '')

# The concept of cleaning these kind of columns is significant here
movies_df[['Actor1', 'Actor2', 'Actor3', 'Actor4']] = movies_df['Actor'].str.split(',', 4, expand=True)
#First 4 directors are considered
movies_df[['Director1', 'Director2', 'Director3', 'Director4', 'DirectorN']] = movies_df['Director'].str.split(',', 4, expand=True)

In [ ]:
for idx, val in movies_df['Director1'].iteritems():
    if val == '': movies_df['Director1'][idx] = 'Unknown'

movies_df['Director1'] = movies_df['Director1'].fillna('Unknown')

In [ ]:
for idx, val in movies_df['Actor1'].iteritems():
    if val == '': movies_df['Actor1'][idx] = 'Unknown'

movies_df['Actor1'] = movies_df['Actor1'].fillna('Unknown')

In [ ]:
movies_df['One-Line'] = movies_df['One-Line'].str.strip('\n')
movies_df['One-Line-Summary'] = movies_df['One-Line'].str.strip()
movies_df['One-Line-Summary'] = movies_df['One-Line-Summary'].str.strip('See full summary »')
movies_df['One-Line-Summary'] = movies_df['One-Line'].str.strip()
#Add a Plot
for idx, val in movies_df['One-Line-Summary'].iteritems():
    if val == 'Add a Plot' or '':
        movies_df['One-Line-Summary'][idx] = 'Unknown'
    else:
        blob = TextBlob(val)
        if movies_df['Movies'][idx] == 'The Kominsky Method': print(idx, blob)
        new_text = ', '.join(blob.noun_phrases)
        #if movies_df['Movies'][idx] == 'The Kominsky Method': print(idx, type(new_text))
        if new_text: movies_df['One-Line-Summary'][idx] = new_text
        else: movies_df['One-Line-Summary'][idx] = 'Unknown'

movies_df['One-Line-Summary'] = movies_df['One-Line-Summary'].fillna('Unknown')

101 An aging actor, who long ago enjoyed a brush with fame, makes his living as an acting coach.
6658 Norman and Madelyn get closer as they spend time together in Santa Barbara. When Sandy meets Mindy's boyfriend, he discovers they have a lot in common.
6659 Mindy's not thrilled with her dad and boyfriend's growing friendship. Norman's daughter leaves rehab, returning home while Madelyn is visiting.
6660 Interactions with two women have Sandy contemplating his changing libido. Norman tries to reconcile with both Phoebe and Madelyn.
6665 Sandy urges Norman to make a grand gesture for Madelyn. After Martin experiences a medical episode, Sandy decides to have a complete checkup.
6666 Despite Norman's prodding, Sandy remains reluctant to share some important news with Mindy. Norman and Madelyn come to an understanding.
6667 An accident forces Sandy to come clean with Mindy. After visiting Eileen's grave, Phoebe tries to make amends with Norman, but it doesn't go well.
6668 As Norman contem

In [ ]:
movies_df.columns

Index(['Movies', 'Year', 'Genre', 'Rating', 'One-Line', 'Stars', 'Votes',
       'Runtime', 'Gross (in million $)', 'Mix', 'End_Year', 'Start_Year',
       'Season', 'Type', 'Genre1', 'Genre2', 'Genre3', 'Director', 'Actor',
       'Actor1', 'Actor2', 'Actor3', 'Actor4', 'Director1', 'Director2',
       'Director3', 'Director4', 'DirectorN', 'One-Line-Summary'],
      dtype='object')

In [ ]:
movies_df.sort_values(by=['Gross (in million $)'], ascending=False, inplace=True)
movies_df = movies_df.reset_index(drop=True)
movies_df.head()

,Movies,Year,Genre,Rating,One-Line,Stars,Votes,Runtime,Gross (in million $),Mix,End_Year,Start_Year,Season,Type,Genre1,Genre2,Genre3,Director,Actor,Actor1,Actor2,Actor3,Actor4,Director1,Director2,Director3,Director4,DirectorN,One-Line-Summary
0,Beauty and the Beast,I 2017,"Family, Fantasy, Musical",7.1,"A selfish Prince is cursed to become a monster for the rest of his life, unless he learns to fall in love with a beautiful young woman he keeps prisoner.","Director:\nBill Condon\n| \n Stars:\nEmma Watson, \nDan Stevens, \nLuke Evans, \nJosh Gad",284484,129.0,504.01,I 2017,None,2017,I,,Family,Fantasy,Musical,Bill Condon,"Emma Watson, Dan Stevens, Luke Evans, Josh Gad",Emma Watson,Dan Stevens,Luke Evans,Josh Gad,Bill Condon,None,None,None,None,"selfish prince, beautiful young woman"
1,Finding Dory,2016,"Animation, Adventure, Comedy",7.3,"Friendly but forgetful blue tang Dory begins a search for her long-lost parents, and everyone learns a few things about the real meaning of family along the way.","Directors:\nAndrew Stanton, \nAngus MacLane\n| \n Stars:\nEllen DeGeneres, \nAlbert Brooks, \nEd O'Neill, \nKaitlin Olson",250953,97.0,486.30,2016,None,2016,,,Animation,Adventure,Comedy,"Andrew Stanton, \nAngus MacLane","Ellen DeGeneres, Albert Brooks, Ed O'Neill, Kaitlin Olson",Ellen DeGeneres,Albert Brooks,Ed O'Neill,Kaitlin Olson,Andrew Stanton,\nAngus MacLane,None,None,None,"friendly, blue tang, dory, long-lost parents, real meaning"
2,Captain America: Civil War,2016,"Action, Adventure, Sci-Fi",7.8,Political involvement in the Avengers' affairs causes a rift between Captain America and Iron Man.,"Directors:\nAnthony Russo, \nJoe Russo\n| \n Stars:\nChris Evans, \nRobert Downey Jr., \nScarlett Johansson, \nSebastian Stan",697750,147.0,408.08,2016,None,2016,,,Action,Adventure,Sci-Fi,"Anthony Russo, \nJoe Russo","Chris Evans, Robert Downey Jr., Scarlett Johansson, Sebastian Stan",Chris Evans,Robert Downey Jr.,Scarlett Johansson,Sebastian Stan,Anthony Russo,\nJoe Russo,None,None,None,"political involvement, avengers, america, iron"
3,Jumanji: Welcome to the Jungle,2017,"Action, Adventure, Comedy",6.9,"Four teenagers are sucked into a magical video game, and the only way they can escape is to work together to finish the game.","Director:\nJake Kasdan\n| \n Stars:\nDwayne Johnson, \nKaren Gillan, \nKevin Hart, \nJack Black",337900,119.0,404.52,2017,None,2017,,,Action,Adventure,Comedy,Jake Kasdan,"Dwayne Johnson, Karen Gillan, Kevin Hart, Jack Black",Dwayne Johnson,Karen Gillan,Kevin Hart,Jack Black,Jake Kasdan,None,None,None,None,magical video game
4,Spider-Man,2002,"Action, Adventure, Sci-Fi",7.3,"When bitten by a genetically modified spider, a nerdy, shy, and awkward high school student gains spider-like abilities that he eventually must use to fight evil as a superhero after tragedy befalls his family.","Director:\nSam Raimi\n| \n Stars:\nTobey Maguire, \nKirsten Dunst, \nWillem Dafoe, \nJames Franco",709684,121.0,403.71,2002,None,2002,,,Action,Adventure,Sci-Fi,Sam Raimi,"Tobey Maguire, Kirsten Dunst, Willem Dafoe, James Franco",Tobey Maguire,Kirsten Dunst,Willem Dafoe,James Franco,Sam Raimi,None,None,None,None,"awkward high school student gains spider-like abilities, tragedy befalls"
